In [15]:
!pip install nltk

In [25]:
# Standard Library
from collections import defaultdict
from datetime import datetime
import logging as logger
import ntpath
import time
import re
import os

# Third-party
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
import nltk

# Setup
nltk.download('vader_lexicon')
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [52]:
# Admin messages - useless for sentiment analysis
FILTERS = [
      "<Media omitted>",
      "changed the settings so only admins can edit the group settings",
      "changed this group's icon",
      "changed the group description",
      "pinned a message",
      "added",
      "now an admin",
      "removed",
      "no longer an admin",
      "joined using this group's invite link",
      "left",
      "changed this group's settings to allow only admins to send messages to this group",
      "started a call",
      "changed this group's settings to allow all members to send messages to this group",
      "changed the settings so all members can edit the group settings",
      "created group",
      "Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.",
      "changed their phone number to a new number. Tap to message or add the new number.",
      "was added",
      "changed to",
      "This message was deleted",
      "This group has over 256 members so now only admins can edit the group settings.",
      "New members need admin approval to join this group.",
      "As a member, you can join groups in the community and get admin updatesYour profile is visible to admins",
      "As a member, you can join groups in the community and get admin updates",
      "Your profile is visible to admins",
      "joined from the community",
      "updated the message timer. New messages will disappear from this chat 7 days after they're sent, except when kept.",
]

FILTERS_REGEX = re.compile('|'.join(map(re.escape, FILTERS)))

In [53]:
"""
Validates the chat file line by line.
If a line has no timestamp, it's part of a multi-line message.
Therefore, the content is modified to act as one message.

Args:
  filePath (str): The path to the chat logs file, must be an exported .txt file.

Returns:
  str: The path to the validated file.
"""
def data_validation(filePath):
  fileName = ntpath.basename(filePath).removesuffix(".txt")
  with open(filePath, 'r') as f1, open(f"validated-{fileName}.txt", 'wb+') as f2:
    for line in f1:
      if not FILTERS_REGEX.search(line):
        try:
          candidate_timestamp = datetime.strptime(line.split(" - ")[0].replace(",", ""), "%m/%d/%y %H:%M") # Bottleneck
        except:
          # Bottleneck
          f2.seek(-1, 2)
          f2.truncate()

        f2.write(line.encode())

  f1.close()
  f2.close()
  return os.path.abspath(f"validated-{fileName}.txt")

In [43]:
"""
Analyzes all messages in a text file to calculate sentiment scores.

Args:
  filePath (str): The path to the validated chat logs file.

Returns:
  dict: A dictionary of all chat members and their sentiment scores.
"""
def sentiment_analysis(filePath):
  members_sentiment = defaultdict(list)
  analyzer = SentimentIntensityAnalyzer()

  for line in open(filePath, 'r'):
    # Get message components
    try:
      initial_split = line.split(" - ")
      timestamp = datetime.strptime(initial_split[0].replace(",", ""), "%m/%d/%y %H:%M")
      content = initial_split[1]
      content_split = content.split(":")
      msg_sender = content_split[0]
      msg = content_split[1]
    except Exception as e:
      print(f"ERROR LINE: {line}")
      logger.error(e)
      break

    # Acquire compounded sentiment score
    score = analyzer.polarity_scores(msg)["compound"]

    if members_sentiment[msg_sender]:
      members_sentiment[msg_sender].append((members_sentiment[msg_sender][-1][0] + score, timestamp))
    else:
      members_sentiment[msg_sender] = [(0.0 + score, timestamp)]

  return members_sentiment

In [20]:
"""
Takes sentiment_analysis()'s resulting dictionary and creates a sentiment chart for each member.
Resulting directory can be downloaded.

Args:
  sentiment_dictionary (dict): A dictionary of all chat members and their sentiment scores.

Returns:
  None
"""
def get_charts(sentiment_dictionary):
  os.makedirs(f"{os.getcwd()}/sentiment_chart_images/", exist_ok = True)
  directory_path = os.path.abspath("sentiment_chart_images")

  for group_member in sentiment_dictionary:
    y = np.array([score for score, _ in sentiment_dictionary[group_member]])
    x = np.array([timestamp for _, timestamp in sentiment_dictionary[group_member]])

    plt.plot(x, y)
    plt.xlabel("Dates")
    plt.ylabel("Positivity Score")
    plt.title(f"{group_member}'s positivity score")
    plt.gcf().autofmt_xdate(rotation = 45)

    plt.savefig(f"{directory_path}/{group_member}-sentiment.png", bbox_inches='tight')
    plt.clf()

In [36]:
test_file = "/content/chats.txt"
data_validation(test_file)

'/content/validated-chats.txt'

In [ ]:
validated_file = "/content/validated-chats.txt"
dictt = sentiment_analysis(validated_file)

In [ ]:
import shutil

shutil.make_archive("sentiment_chart_images", 'zip', "sentiment_chart_images")

In [54]:
def prun_data_validation(no_times):
  for _ in range(no_times):
    data_validation(test_file)

%prun prun_data_validation(10)